In [1]:
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
import numpy as np
import pickle
import umap.umap_ as umap
from NFLUtils import NFLUtils
nfl_utils = NFLUtils()

# 1. Import as dataframe, create additional columns
Import dataset, set df as the original reference dataset

In [2]:
# Import cont_cols and track_cols from nfl_utils
cont_cols = nfl_utils.cont_cols
track_cols = nfl_utils.track_cols

df = pd.read_csv('footballData/combined.csv', index_col=False, low_memory=False)
df = df.sort_values(by='Date')

# Create the H_Won (Y) column
df['H_Won'] = np.where(df['H_Final'] > df['V_Final'], 1.0, 0.0)

# Combine passing & rushing TD for UMAP
df['H_passing_rushing_td'] = df['H_passing_td'] + df['H_rushing_td']
df['V_passing_rushing_td'] = df['V_passing_td'] + df['V_rushing_td']

# Create the H_final_allowed, V_final_allowed columns
df['H_Final_Allowed'] = df['V_Final']
df['V_Final_Allowed'] = df['H_Final']

# print(f'df after perf set removed: {df.shape}')
# print(f'df perf set size {test_performance_df.shape}')

# for i in range(0,len(df.columns)):
#    print(f"{df.columns[i]}\t\t\t\t{df.dtypes[i]}")

# 2. Check and remove missing data
from https://www.kaggle.com/code/gpreda/santander-eda-and-prediction

In [3]:
def missing_data(data):
    total = data.isnull().sum()
    percent = (data.isnull().sum()/data.isnull().count()*100)
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    return(np.transpose(tt))

In [4]:
pd.set_option('display.max_columns', None)
df.dropna()
missing_data(df)

,Season,Date,Home_Team,H_Q1,H_Q2,H_Q3,H_Q4,H_OT,H_Final,Visitor_Team,V_Q1,V_Q2,V_Q3,V_Q4,V_OT,V_Final,H_First_Downs,V_First_Downs,H_Rush,V_Rush,H_Yds,V_Yds,H_TDs,V_TDs,H_Cmp,V_Cmp,H_Att,V_Att,H_Yd,V_Yd,H_TD,V_TD,H_INT,V_INT,H_Sacked,V_Sacked,H_Sacked_Yards,V_Sacked_Yards,H_Net_Pass_Yards,V_Net_Pass_Yards,H_Total_Yards,V_Total_Yards,H_Fumbles,V_Fumbles,H_Lost,V_Lost,H_Turnovers,V_Turnovers,H_Penalties,V_Penalties,H_Penalties_Yards,V_Penalties_Yards,H_Third_Down_Conv,V_Third_Down_Conv,H_Fourth_Down_Conv,V_Fourth_Down_Conv,H_Time_of_Possession,V_Time_of_Possession,H_passing_att,H_passing_cmp,H_passing_int,H_passing_lng,H_passing_sk,H_passing_td,H_passing_yds,H_receiving_lng,H_receiving_td,H_receiving_yds,H_rushing_att,H_rushing_lng,H_rushing_td,H_rushing_yds,V_passing_att,V_passing_cmp,V_passing_int,V_passing_lng,V_passing_sk,V_passing_td,V_passing_yds,V_receiving_lng,V_receiving_td,V_receiving_yds,V_rushing_att,V_rushing_lng,V_rushing_td,V_rushing_yds,H_def_interceptions_int,H_def_interceptions_lng,H_def_interceptions_td,H_def_interceptions_yds,H_fumbles_ff,H_fumbles_fr,H_fumbles_td,H_fumbles_yds,H_sk,H_tackles_ast,H_tackles_comb,H_tackles_solo,V_def_interceptions_int,V_def_interceptions_lng,V_def_interceptions_td,V_def_interceptions_yds,V_fumbles_ff,V_fumbles_fr,V_fumbles_td,V_fumbles_yds,V_sk,V_tackles_ast,V_tackles_comb,V_tackles_solo,H_kick_returns_lng,H_kick_returns_rt,H_kick_returns_td,H_kick_returns_yds,H_punt_returns_lng,H_punt_returns_ret,H_punt_returns_td,H_punt_returns_yds,V_kick_returns_lng,V_kick_returns_rt,V_kick_returns_td,V_kick_returns_yds,V_punt_returns_lng,V_punt_returns_ret,V_punt_returns_td,V_punt_returns_yds,H_punting_lng,H_punting_pnt,H_punting_yds,H_scoring_fga,H_scoring_fgm,H_scoring_xpa,H_scoring_xpm,V_punting_lng,V_punting_pnt,V_punting_yds,V_scoring_fga,V_scoring_fgm,V_scoring_xpa,V_scoring_xpm,H_halftime_odds,V_halftime_odds,H_start_odds,V_start_odds,H_Won,H_passing_rushing_td,V_passing_rushing_td,H_Final_Allowed,V_Final_Allowed
Total,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,31,29,31,29,27,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Percent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.388034,0.362999,0.388034,0.362999,0.337965,0.31293,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Types,int64,object,object,int64,int64,int64,int64,int64,int64,object,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,object,object,object,object,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,

In the above table, we see the following sets of columns that contain empty values:

In [5]:
columns_with_empty_values = df.columns[df.isnull().sum() > 0].tolist()
print(columns_with_empty_values)

['H_Third_Down_Conv', 'V_Third_Down_Conv', 'H_Fourth_Down_Conv', 'V_Fourth_Down_Conv', 'H_Time_of_Possession', 'V_Time_of_Possession']


Since some of these columns are strings, replace them with NaN and drop them via df.dropna

There are so few of them that we don't really need to worry about the missing games interfering with the overall prediction

In [6]:
for column in columns_with_empty_values:
    df[column].replace('', np.nan, inplace=True)
df.dropna(subset=columns_with_empty_values, inplace=True)
missing_data(df)

/var/folders/k0/7b4qgkdx2vb9ml4ktdckf0hc0000gn/T/ipykernel_3960/2769842864.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].replace('', np.nan, inplace=True)


,Season,Date,Home_Team,H_Q1,H_Q2,H_Q3,H_Q4,H_OT,H_Final,Visitor_Team,V_Q1,V_Q2,V_Q3,V_Q4,V_OT,V_Final,H_First_Downs,V_First_Downs,H_Rush,V_Rush,H_Yds,V_Yds,H_TDs,V_TDs,H_Cmp,V_Cmp,H_Att,V_Att,H_Yd,V_Yd,H_TD,V_TD,H_INT,V_INT,H_Sacked,V_Sacked,H_Sacked_Yards,V_Sacked_Yards,H_Net_Pass_Yards,V_Net_Pass_Yards,H_Total_Yards,V_Total_Yards,H_Fumbles,V_Fumbles,H_Lost,V_Lost,H_Turnovers,V_Turnovers,H_Penalties,V_Penalties,H_Penalties_Yards,V_Penalties_Yards,H_Third_Down_Conv,V_Third_Down_Conv,H_Fourth_Down_Conv,V_Fourth_Down_Conv,H_Time_of_Possession,V_Time_of_Possession,H_passing_att,H_passing_cmp,H_passing_int,H_passing_lng,H_passing_sk,H_passing_td,H_passing_yds,H_receiving_lng,H_receiving_td,H_receiving_yds,H_rushing_att,H_rushing_lng,H_rushing_td,H_rushing_yds,V_passing_att,V_passing_cmp,V_passing_int,V_passing_lng,V_passing_sk,V_passing_td,V_passing_yds,V_receiving_lng,V_receiving_td,V_receiving_yds,V_rushing_att,V_rushing_lng,V_rushing_td,V_rushing_yds,H_def_interceptions_int,H_def_interceptions_lng,H_def_interceptions_td,H_def_interceptions_yds,H_fumbles_ff,H_fumbles_fr,H_fumbles_td,H_fumbles_yds,H_sk,H_tackles_ast,H_tackles_comb,H_tackles_solo,V_def_interceptions_int,V_def_interceptions_lng,V_def_interceptions_td,V_def_interceptions_yds,V_fumbles_ff,V_fumbles_fr,V_fumbles_td,V_fumbles_yds,V_sk,V_tackles_ast,V_tackles_comb,V_tackles_solo,H_kick_returns_lng,H_kick_returns_rt,H_kick_returns_td,H_kick_returns_yds,H_punt_returns_lng,H_punt_returns_ret,H_punt_returns_td,H_punt_returns_yds,V_kick_returns_lng,V_kick_returns_rt,V_kick_returns_td,V_kick_returns_yds,V_punt_returns_lng,V_punt_returns_ret,V_punt_returns_td,V_punt_returns_yds,H_punting_lng,H_punting_pnt,H_punting_yds,H_scoring_fga,H_scoring_fgm,H_scoring_xpa,H_scoring_xpm,V_punting_lng,V_punting_pnt,V_punting_yds,V_scoring_fga,V_scoring_fgm,V_scoring_xpa,V_scoring_xpm,H_halftime_odds,V_halftime_odds,H_start_odds,V_start_odds,H_Won,H_passing_rushing_td,V_passing_rushing_td,H_Final_Allowed,V_Final_Allowed
Total,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Percent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Types,int64,object,object,int64,int64,int64,int64,int64,int64,object,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,object,object,object,object,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64


## 3. Feature engineering Time_of_Possession & *_Down_Conv
Time_of_Possession from %M:%S to %M
Third & fourth down conversion from attempts-total to percentage

In [7]:
for team in ["H", "V"]:
    # Time of possessiion conversion to minutes only
    df[f"{team}_Time_of_Possession"] = pd.to_datetime(df[f"{team}_Time_of_Possession"], format="%M:%S").dt.minute
    
    # Conversion from attempts-total to percentage
    df[f'{team}_Third_Down_Conv'] = df[f'{team}_Third_Down_Conv'].apply(
        lambda x: 0 if int(x.split('-')[1]) == 0 else (int(x.split('-')[0]) / int(x.split('-')[1])) * 100
    )
    df[f'{team}_Fourth_Down_Conv'] = df[f'{team}_Fourth_Down_Conv'].apply(
        lambda x: 0 if int(x.split('-')[1]) == 0 else (int(x.split('-')[0]) / int(x.split('-')[1])) * 100
    )

# 4. Combine similar columns
Here we combine the 8 kick_returns & punt_returns columns into 4 kick_punt returns columns. We will calculate maximum between each longest punt (lng) columns and will add the rest

In [8]:
# These columns will be dropped and ignored after everything is calculated
drop_columns = [
    # Removed b/c correlation matrix similarity
    "H_passing_yds", "V_passing_yds",
    "H_receiving_yds", "V_receiving_yds",    
    "H_def_interceptions_lng", "V_def_interceptions_lng",
    "H_receiving_td", "V_receiving_td",
    
    # Removed b/c unimportant (based on domain knowledge)
    "H_punting_lng", "V_punting_lng",
    
    # Removed b/c replaced with meta-feature
    "H_punting_yds", "V_punting_yds",
    "H_scoring_fgm", "V_scoring_fgm",
    "H_scoring_xpm", "V_scoring_xpm"
]


# These columns will be added to create the new (key) column
# The columns composing the key column will be dropped
# { <New column name> : [<combine column names>, ...] }
combine_dict_add = {
    "kick_punt_returns_rt": ["kick_returns_rt", "punt_returns_ret"],
    "kick_punt_returns_td": ["kick_returns_td", "punt_returns_td"],
    "kick_punt_returns_yds": ["kick_returns_yds", "punt_returns_yds"]
}

# These columns will be divided. None will be dropped unless in drop_columns
# { <New column name> : [<numerator column name>, <denominator column name>] }
combine_dict_divide = {
    "scoring_fgp": ["scoring_fgm", "scoring_fga"],
    "scoring_xpp": ["scoring_xpm", "scoring_xpa"],
    "punting_avg": ["punting_yds", "punting_pnt"],
}

# Perform column based operations (max, add, divide)
for team in ["H", "V"]:
    # For the longest kick/punt, get the max & drop columns
    df[f'{team}_kick_punt_returns_lng'] = np.maximum(df[f'{team}_kick_returns_lng'], df[f'{team}_punt_returns_lng'])
    df.drop([f'{team}_kick_returns_lng', f'{team}_punt_returns_lng'], axis=1, inplace=True)

    # -------- Columns to add --------
    for (key, value) in combine_dict_add.items():
        df[f'{team}_{key}'] = df[f'{team}_{value[0]}'] + df[f'{team}_{value[1]}']
        df.drop([f'{team}_{value[0]}', f'{team}_{value[1]}'], axis=1, inplace=True)

    # -------- Columns to divide --------
    for (key, value) in combine_dict_divide.items():
        df[f'{team}_{key}'] = np.where(df[f'{team}_{value[1]}'] > 0, df[f'{team}_{value[0]}']/df[f'{team}_{value[1]}'], 0)

        
# Drop columns
df.drop(drop_columns, axis=1, inplace=True)
print(len(df.columns))
print(list(df.columns))

131
['Season', 'Date', 'Home_Team', 'H_Q1', 'H_Q2', 'H_Q3', 'H_Q4', 'H_OT', 'H_Final', 'Visitor_Team', 'V_Q1', 'V_Q2', 'V_Q3', 'V_Q4', 'V_OT', 'V_Final', 'H_First_Downs', 'V_First_Downs', 'H_Rush', 'V_Rush', 'H_Yds', 'V_Yds', 'H_TDs', 'V_TDs', 'H_Cmp', 'V_Cmp', 'H_Att', 'V_Att', 'H_Yd', 'V_Yd', 'H_TD', 'V_TD', 'H_INT', 'V_INT', 'H_Sacked', 'V_Sacked', 'H_Sacked_Yards', 'V_Sacked_Yards', 'H_Net_Pass_Yards', 'V_Net_Pass_Yards', 'H_Total_Yards', 'V_Total_Yards', 'H_Fumbles', 'V_Fumbles', 'H_Lost', 'V_Lost', 'H_Turnovers', 'V_Turnovers', 'H_Penalties', 'V_Penalties', 'H_Penalties_Yards', 'V_Penalties_Yards', 'H_Third_Down_Conv', 'V_Third_Down_Conv', 'H_Fourth_Down_Conv', 'V_Fourth_Down_Conv', 'H_Time_of_Possession', 'V_Time_of_Possession', 'H_passing_att', 'H_passing_cmp', 'H_passing_int', 'H_passing_lng', 'H_passing_sk', 'H_passing_td', 'H_receiving_lng', 'H_rushing_att', 'H_rushing_lng', 'H_rushing_td', 'H_rushing_yds', 'V_passing_att', 'V_passing_cmp', 'V_passing_int', 'V_passing_lng', 

In [9]:
# df['Date'].values
df['Season'].values

array([1995, 1995, 1995, ..., 2024, 2024, 2024])

# 5. Create a dict to track the track_cols array
create another dict to track previous games for each team during the year

In [10]:
track_dict = {}

for row in df.itertuples():
    # year = row.Date.split('-')[0]
    year = row.Season
    home_team = row.Home_Team
    visitor_team = row.Visitor_Team
    
    # Home or visitor team has < minimum_window total games
    for col in track_cols:
        home_column_name = f'{year}_{home_team}_{col}'
        visitor_column_name = f'{year}_{visitor_team}_{col}'
        
        # Home team
        home_col = col if col == 'Date' else 'H_' + col
        if home_column_name in track_dict:
            track_dict[home_column_name].append(getattr(row, home_col))
        else:
            track_dict[home_column_name] = [getattr(row, home_col)]
        
        # Visitor team
        visitor_col = col if col == 'Date' else 'V_' + col
        if visitor_column_name in track_dict:
            track_dict[visitor_column_name].append(getattr(row, visitor_col))
        else:
            track_dict[visitor_column_name] = [getattr(row, visitor_col)]

In [11]:
track_dict['1999_NWE_Rush']

[27, 121, 57, 35, 19, 122, 133, 43, 188, 19, 25, 63, 20, 27, 168, 119]

# 6. Use track_dict to reformat df
- enforce minimum_window, update df or drop row
- If either team have played less than _minimum_window_ games, ignore the current game and drop the row
- Otherwise, get the EMA of all of the previous games in the current season for both teams. Use this to calculatee most of the columns
- After completion, each row will contain data up to (not including) the current game

This allows us to access the current data of game n taking into account all previous games not including game n. Knowing this, we can train a predictive model on each row without data leakage provided the data is offered sequentially.

In [12]:
minimum_window = 4
ema_span = 7
print(df.shape)

indices_to_drop = []
current_count = 0

for row in df.itertuples():
    if current_count % 300 == 0:
        print(f'{current_count}/{df.shape[0]}')
    current_count = current_count + 1
    index = row.Index
    # year = row.Date.split('-')[0]
    year = row.Season
    home_team = row.Home_Team
    visitor_team = row.Visitor_Team
    # Home team min window
    home_date_column = f'{year}_{home_team}_Date'
    visitor_date_column = f'{year}_{visitor_team}_Date'

    # Current row is older than Home team at min_window
    if len(track_dict[home_date_column]) > minimum_window and row.Date <= track_dict[home_date_column][minimum_window]:
        indices_to_drop.append(index)
        continue
    # Current row is older than Visitor team at min_window
    if len(track_dict[visitor_date_column]) > minimum_window and row.Date <= track_dict[visitor_date_column][minimum_window]:
        indices_to_drop.append(index)
        continue

    home_date_index = track_dict[home_date_column].index(row.Date)
    visitor_date_index = track_dict[visitor_date_column].index(row.Date)
    # print(f'H: {home_date_index} V: {visitor_date_index}')

    # Update df to have average for each track_cols (Ignoring 'Date', 'datediff' the 1-2nd item)
    for col in track_cols:
        if col in ['Date', 'datediff', 'start_odds', 'halftime_odds']:
            continue
        else:
            # Convert to float
            if df.dtypes['H_' + col] == 'int64' or df.dtypes['V_' + col] == 'int64':
                df = df.astype({f'H_{col}': 'float64', f'V_{col}': 'float64'})
            # Update df to have average for each track_cols (Ignoring 'Date', 'datediff' the 1-2nd item)
            # Update home
            home_col_list = track_dict[f'{year}_{home_team}_{col}'][:home_date_index-1]
            dataframe_val = pd.DataFrame({'value': home_col_list})
            ema = dataframe_val['value'].ewm(span=min(ema_span, len(home_col_list)), adjust=True).mean().iloc[-1]
            df.at[index, 'H_' + col] = ema

            # Update Visitor
            visitor_col_list = track_dict[f'{year}_{visitor_team}_{col}'][:visitor_date_index-1]
            dataframe_val = pd.DataFrame({'value': visitor_col_list})
            ema = dataframe_val['value'].ewm(span=min(minimum_window, len(visitor_col_list)), adjust=False).mean().iloc[-1]
            df.at[index, 'V_' + col] = ema


    # --------------------------------------------------- 
    # ------------------ Custom Columns -----------------
    # --------------------------------------------------- 
    #
    
    # 1. Add variant of Bill James pythagorean expectation (NFL).
    # Recent games weighted more heavily since 'Final' columns not excluded from the above loop
    home_points_for = sum(track_dict[f'{year}_{home_team}_Final'][:home_date_index-1])
    home_points_against = sum(track_dict[f'{year}_{home_team}_Final_Allowed'][:home_date_index-1])
    df.at[index, 'H_pythagorean'] = home_points_for**2.37 / (home_points_for**2.37 + home_points_against**2.37)

    visitor_points_for = sum(track_dict[f'{year}_{visitor_team}_Final'][:visitor_date_index-1])
    visitor_points_against = sum(track_dict[f'{year}_{visitor_team}_Final_Allowed'][:visitor_date_index-1])
    df.at[index, 'V_pythagorean'] = visitor_points_for**2.37 / (visitor_points_for**2.37 + visitor_points_against**2.37)            
            
        
    # 2. Add num days since last game for home, visitor
    df.at[index, f'H_datediff'] = 0
    if home_date_index > 0:
        current_game_date = datetime.strptime(track_dict[home_date_column][home_date_index], "%Y-%m-%d")
        previous_game_date = datetime.strptime(track_dict[home_date_column][home_date_index-1], "%Y-%m-%d")
        game_diff = int((current_game_date - previous_game_date).days)
        # print(f'{current_game_date} minus {previous_game_date} is {game_diff}')
        df.at[index, f'H_datediff'] = game_diff
    
    df.at[index, f'V_datediff'] = 0
    if visitor_date_index > 0:
        current_game_date = datetime.strptime(track_dict[visitor_date_column][visitor_date_index], "%Y-%m-%d")
        previous_game_date = datetime.strptime(track_dict[visitor_date_column][visitor_date_index-1], "%Y-%m-%d")
        game_diff = int((current_game_date - previous_game_date).days)
        # print(f'{current_game_date} minus {previous_game_date} is {game_diff}')
        df.at[index, f'V_datediff'] = game_diff
        
df.drop(indices_to_drop, inplace=True)

# Add custom metrics to track_cols so it creates the difference (D_) column
track_cols.append('datediff')
track_cols.append('pythagorean')
for col in track_cols[1:]:
    cont_cols.append('D_' + col)
    df['D_' + col] = (df['H_' + col] - df['V_' + col]).round(3) # Round to 3 sig figs

print(track_cols)
track_cols.pop()


print(df.shape)

(7957, 131)
0/7957


/var/folders/k0/7b4qgkdx2vb9ml4ktdckf0hc0000gn/T/ipykernel_3960/4084338832.py:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '25.53676470588235' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df.at[index, 'H_' + col] = ema
/var/folders/k0/7b4qgkdx2vb9ml4ktdckf0hc0000gn/T/ipykernel_3960/4084338832.py:53: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '27.160000000000004' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df.at[index, 'V_' + col] = ema
/var/folders/k0/7b4qgkdx2vb9ml4ktdckf0hc0000gn/T/ipykernel_3960/4084338832.py:65: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. 

300/7957
600/7957
900/7957
1200/7957
1500/7957
1800/7957
2100/7957
2400/7957
2700/7957
3000/7957
3300/7957
3600/7957
3900/7957
4200/7957
4500/7957
4800/7957
5100/7957
5400/7957
5700/7957
6000/7957
6300/7957
6600/7957
6900/7957
7200/7957
7500/7957
7800/7957
['Date', 'First_Downs', 'Rush', 'Yds', 'TDs', 'Cmp', 'Att', 'Yd', 'TD', 'INT', 'Sacked', 'Sacked_Yards', 'Net_Pass_Yards', 'Total_Yards', 'Fumbles', 'Lost', 'Turnovers', 'Penalties', 'Third_Down_Conv', 'Fourth_Down_Conv', 'Time_of_Possession', 'passing_att', 'passing_cmp', 'passing_int', 'passing_lng', 'passing_sk', 'passing_td', 'receiving_lng', 'rushing_att', 'rushing_lng', 'rushing_td', 'rushing_yds', 'passing_rushing_td', 'def_interceptions_int', 'def_interceptions_td', 'def_interceptions_yds', 'fumbles_ff', 'fumbles_fr', 'fumbles_td', 'fumbles_yds', 'sk', 'tackles_ast', 'tackles_comb', 'tackles_solo', 'kick_punt_returns_lng', 'kick_punt_returns_rt', 'kick_punt_returns_td', 'kick_punt_returns_yds', 'punting_pnt', 'punting_avg', '

/var/folders/k0/7b4qgkdx2vb9ml4ktdckf0hc0000gn/T/ipykernel_3960/4084338832.py:96: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['D_' + col] = (df['H_' + col] - df['V_' + col]).round(3) # Round to 3 sig figs
/var/folders/k0/7b4qgkdx2vb9ml4ktdckf0hc0000gn/T/ipykernel_3960/4084338832.py:96: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['D_' + col] = (df['H_' + col] - df['V_' + col]).round(3) # Round to 3 sig figs
/var/folders/k0/7b4qgkdx2vb9ml4ktdckf0hc0000gn/T/ipykernel_3960/4084338832.py:96: PerformanceWarning: DataFrame is

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
Name: value, dtype: float64
ema is: 0.0
dataframe_val
    value
0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
ema
0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
Name: value, dtype: float64
ema is: 0.0
dataframe_val
    value
0     6.0
1     6.0
2     7.0
3     8.0
4     6.0
5     8.0
6     7.0
7     4.0
8     7.0
9     5.0
10    4.0
ema
0     6.000000
1     6.000000
2     6.432432
3     7.005714
4     6.676056
5     7.078705
6     7.055998
7     6.207003
8     6.421346
9     6.044805
10    5.511061
Name: value, dtype: float64
ema is: 5.511061180830124
dataframe_val
    value
0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
ema
0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0

KeyboardInterrupt: 

# UMAP
Remove the performance set, add UMAP columns _kick_punt_umap_dim_1_ and _kick_punt_umap_dim_2_

Save the UMAP model as a pickle file

In [13]:
# df["D_passing_rushing_td"].head()
print((df["D_kick_punt_returns_td"] > 0).sum())
print((df["D_kick_punt_returns_td"] == 0).sum())

# Remove the performance set
test_performance_size = 200
test_performance_df = df[df.shape[0]-test_performance_size:]
df = df[:df.shape[0]-test_performance_size]

1461
2714


In [14]:
import seaborn as sns
from sklearn.preprocessing import StandardScaler


# Explosive play indicators
return_game_explosives = ["D_kick_punt_returns_lng", "D_kick_punt_returns_rt", "D_kick_punt_returns_yds"]
return_game_df = df[return_game_explosives]
scaled_return_game_df = StandardScaler().fit_transform(return_game_df)

reducer = umap.UMAP(random_state=42, n_neighbors=40, min_dist=0.1)
embedding = reducer.fit_transform(scaled_return_game_df)
print(embedding.shape)

# Create the two new columns, drop the 4
df['kick_punt_umap_dim_1'] = embedding[:,0]
df['kick_punt_umap_dim_2'] = embedding[:,1]
df.drop(return_game_explosives + ['D_kick_punt_returns_td'], axis=1, inplace=True)

# Save UMAP model as pickle
filename = "kick_punt_umap.sav"
pickle.dump(reducer, open(filename, 'wb'))

df = pd.concat([df, test_performance_df], axis=0)

# ------------- UMAP explore parameters -------------
# start_neighbors = 5
#while start_neighbors < 41:
#    start_dist = 0.1
#    while start_dist < 0.5:
#        reducer = umap.UMAP(random_state=42, n_neighbors=start_neighbors, min_dist=0.1)
#        embedding = reducer.fit_transform(scaled_return_game_df)
#        # Create a new figure for each plot
#        plt.scatter(
#            embedding[:, 0],
#            embedding[:, 1],
#            #c=df["D_kick_punt_returns_td"].astype(float),
#            #c=df["H_Won"],
#            c=df["D_Final"],            
#            cmap='Spectral',
#            s=5
#        )
#        plt.gca().set_aspect('equal', 'datalim')
#        plt.title(f'UMAP with n_neighbors={start_neighbors} start_dist={start_dist}')
#        plt.show()
#        start_dist = start_dist * 2
#    start_neighbors = start_neighbors * 2


/Users/forbesjon2/anaconda3/envs/pytorchenv/lib/python3.9/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


(5305, 2)


/var/folders/k0/7b4qgkdx2vb9ml4ktdckf0hc0000gn/T/ipykernel_3960/685224282.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['kick_punt_umap_dim_1'] = embedding[:,0]
/var/folders/k0/7b4qgkdx2vb9ml4ktdckf0hc0000gn/T/ipykernel_3960/685224282.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['kick_punt_umap_dim_2'] = embedding[:,1]


In [15]:
df.to_csv(f'./footballData/CombinedSlidingWindow{minimum_window}.csv')

In [16]:
print(len(cont_cols))

111
